### Challenge - Attrapez-les tous

Challenge dans le cadre de la formation Data Analyst de la Wild Code School

![ ](https://storage.googleapis.com/quest_editor_uploads/ziGVpdPyvm1HMWm1ILdbCXW5tH8Hlg68.jpg)



Un champion du monde Pokémon (oui, cela existe) a entendu parler de tes connaissances en Machine Learning et demande ton expertise.

Le champion a l'habitude d'utiliser les Pokemon suivants : [Mewtwo](https://bulbapedia.bulbagarden.net/wiki/Mewtwo_(Pokémon)), [Lugia](https://bulbapedia.bulbagarden.net/wiki/Lugia_(Pokémon)), [Rayquaza](https://bulbapedia.bulbagarden.net/wiki/Rayquaza_(Pokémon)), [Giratina](https://bulbapedia.bulbagarden.net/wiki/Giratina_(Pokémon)), [Dialga](https://bulbapedia.bulbagarden.net/wiki/Dialga_(Pokémon)), et [Palkia](https://bulbapedia.bulbagarden.net/wiki/Palkia_(Pokémon)). Mais pour sa prochaine compétition, il est interdit d'utiliser un Pokémon légendaire. Un pokemon est soit légendaire, soit non-légendaire. La dernière colonne du dataset indique True ou False suivant les cas.
Le champion devra donc remplacer ses Pokemon légendaires préférés par d'autres Pokémons non-légendaires, avec des caractéristiques similaires. C'est là que tu interviens !

Pour l'aider, suis les étapes suivantes :

1. Importe le [dataset pokemon.csv](https://raw.githubusercontent.com/murpi/wilddata/master/pokemon.csv). Chaque ligne représente un pokemon, avec ses différentes caractéristiques (attaque, défense, vitesse, etc...), ainsi que la colonne "légendaire".
2. Entraine l'algorithme NN en utilisant l'ensemble des colonnes numériques en variables explicatives `X`.

```
Pas besoin de traintestsplit dans ce cas précis : en effet, sinon tu chercheras les voisins sur un sous-échantillon. Alors qu'il y a peut-être des voisins plus proche.Pas besoin de predict ou de score non plus : nous calculons uniquement des distances.
```

1. Utilise la fonction `kneighbors()` pour trouver le ou les Pokémons le(s) plus proche(s) de chacun des Pokémon légendaires que le champion a l'habitude d'utiliser.
2. Fais des recommandations de Pokémon à utiliser par le champion pour sa prochaine compétition, en lui priorisant par distance. Evidemment, ta recommandation ne doit pas indiquer un autre pokemon légendaire, même si ce dernier est très proche...

In [1]:
#Importation du nécessaire :)
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
df_pokemon = pd.read_csv('https://raw.githubusercontent.com/murpi/wilddata/master/pokemon.csv')

In [76]:
df_pokemon.to_csv('df_pokemon.csv')

Visualisation du DF pour comprendre ce sur quoi nous travaillons 

In [128]:
df_pokemon.head()

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


Recherche de valeurs NaN

In [4]:
df_pokemon.isna().any()

#             False
Name           True
Type 1        False
Type 2         True
HP            False
Attack        False
Defense       False
Sp. Atk       False
Sp. Def       False
Speed         False
Generation    False
Legendary     False
dtype: bool

In [15]:
df_pokemon.isna().sum()

#               0
Name            1
Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [18]:
#Je supprime le seul Pokemon qui n'a pas de nom. 
df_pokemon.dropna(subset = ["Name"], inplace=True)

In [19]:
#Dans ce cas particulier, je ne modifie pas la colonne Type2 car je n'en ai pas besoin. 
df_pokemon.isna().sum()

#               0
Name            0
Type 1          0
Type 2        385
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

## Maintenant je peux commencer avec le ML

In [125]:
#Je sélectionne les colonnes que j'utiliserai pour le modèle NearestNeighbors
X = df_pokemon[['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed','Generation']]
distanceKNN = NearestNeighbors(n_neighbors=10).fit(X)

In [130]:
df_pokemon[df_pokemon['Name'] == 'Mewtwo']

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
162,163,Mewtwo,Psychic,NaN,106,110,90,154,90,130,1,True


In [89]:
#Premier test pour vérifier si tout est bien réglé et pour vérifier le premier résultat. 
neighbors = distanceKNN.kneighbors([[106,110,90,154,90,130,1]])
neighbors

(array([[ 0.        , 41.34005322, 42.22558466]]),
 array([[161, 550, 247]], dtype=int64))

In [131]:
#Je cherche le Pokemo le plus proche de celui qui a été pris comme référence. 
df_pokemon.loc[neighbors[1][0][0]]

#                   162
Name          Dragonite
Type 1           Dragon
Type 2           Flying
HP                   91
Attack              134
Defense              95
Sp. Atk             100
Sp. Def             100
Speed                80
Generation            1
Legendary         False
Name: 161, dtype: object

#### Premier jet pour la recommandation des pokémos à utiliser qui ne sont pas légendaires. 

In [139]:
neighbors3 = distanceKNN.kneighbors(df_pokemon.loc[df_pokemon['Name'] == "Mewtwo",['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed','Generation']])
pokemon_recommandes  = df_pokemon.loc[neighbors3[1][0]][['Name','Legendary']][:5]

mask = pokemon_recommandes['Legendary']==False
print(pokemon_recommandes[mask])


          Name  Legendary
161  Dragonite      False
247   Houndoom      False
274   Sceptile      False


In [141]:
#Loop, pour recommander le pokémon non-légendaire le plus proche. 
lista= [ "Mewtwo", "Lugia", "Rayquaza", "Giratina Altered Forme","Giratina Origin Forme","Dialga", "Palkia"]

for i in lista:
    neighbors3 = distanceKNN.kneighbors(df_pokemon.loc[df_pokemon['Name'] == i,['HP','Attack','Defense','Sp. Atk','Sp. Def','Speed','Generation']])
    pokemon_recommandes = df_pokemon.loc[neighbors3[1][0]][['Name','Legendary']][:10]
    mask = pokemon_recommandes['Legendary']==False
    
    print("")
    print("Instead of {}, you can use: ".format(i))
    print('..................................')
    print(pokemon_recommandes[mask])
    


Instead of Mewtwo, you can use: 
..................................
             Name  Legendary
161     Dragonite      False
247      Houndoom      False
274      Sceptile      False
23   Mega Pidgeot      False
548       Manaphy      False
695      Zweilous      False

Instead of Lugia, you can use: 
..................................
               Name  Legendary
268  Mega Tyranitar      False
536       Mow Rotom      False
155         Snorlax      False
3     Mega Venusaur      False

Instead of Rayquaza, you can use: 
..................................
         Name  Legendary
791   Noivern      False
278  Blaziken      False

Instead of Giratina Altered Forme, you can use: 
..................................
       Name  Legendary
547  Phione      False

Instead of Giratina Origin Forme, you can use: 
..................................
        Name  Legendary
791  Noivern      False

Instead of Dialga, you can use: 
..................................
                 Name  Lege